<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [11]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
import pyLDAvis
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import os

<h2>Creación del Corpus</h2>

In [ ]:
corpus_path = "cyberbullying_corpus/"
#corpus_path = "corpus_lda/corpus_lda.lda_c"
#dictionary_path ="corpus_lda/corpus_lda.dict"

In [ ]:
#Remove keywords in cyberbullying from stopwords list
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

#Create the corpus
for filename in os.listdir(corpus_path):
    with open(corpus_path+filename) as file:
        tokens = []
        for line in file:
            for sym in puncts:
                line = line.replace(sym," ")
            for num in digits:
                line = line.replace(num," ")

            tokens = tokens + [word for word in line.lower().split() if word not in stopword_list]
        corpus.append(tokens)

#Remove words that appear only once
frequency = defaultdict(int)
for text in corpus:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in corpus]

#Create and save dictionary
dictionary = corpora.Dictionary(texts)
dictionary.save('tmp/cyberbullying_dictionary.dict') #Save the dictionary

#Convert documents to vectors
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus) #Save the corpus

In [ ]:
#print(texts[:10])
print(dictionary)
print(corpus)

In [14]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 14.6 ms
Creating corpus...
CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 119 ms


In [19]:
#Initialize a model
#print('Creating Tfidf model...')
#%time tfidf = models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]

#Create a LDA Model and save it
lda = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda.save('tmp/cyberbullying_ldaModel.lda')

In [ ]:
#Load LDA Model
lda = models.LdaModel.load('tmp/cyberbullying_ldaModel.lda')

In [22]:
#Print Topics
lda.print_topics(2,10)

[(0,
  u'0.079*"you" + 0.044*"your" + 0.027*"like" + 0.019*"look" + 0.012*"it" + 0.009*"he" + 0.007*"looks" + 0.006*"face" + 0.005*"get" + 0.004*"one"'),
 (1,
  u'0.080*"you" + 0.032*"your" + 0.026*"like" + 0.017*"it" + 0.016*"look" + 0.008*"he" + 0.007*"face" + 0.006*"one" + 0.006*"looks" + 0.005*"get"')]